In [ ]:
#Import libraries
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import missingno as msno
from fancyimpute import KNN, IterativeImputer
import seaborn as sns

In [ ]:
#Load datasets
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
SampleSubmit = pd.read_csv("SampleSubmission.csv")
VarDef = pd.read_csv("VariableDefinitions.csv")
riders = pd.read_csv("Riders.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
riders.head()

In [ ]:
#Training data including rider information
mergedtrain = pd.merge(train, riders, how = "left", on = 'Rider Id')

## Exploration

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
useless = ["Order No", "User Id", "Vehicle Type", "Placement - Day of Month", "Placement - Weekday (Mo = 1)", 
           "Placement - Time", 'Confirmation - Day of Month','Confirmation - Weekday (Mo = 1)', 'Confirmation - Time',
           'Arrival at Pickup - Day of Month', 'Arrival at Pickup - Weekday (Mo = 1)', 'Arrival at Pickup - Time',
           'Pickup - Day of Month', 'Arrival at Destination - Day of Month', 'Arrival at Destination - Weekday (Mo = 1)', 
           'Arrival at Destination - Time', 'Precipitation in millimeters', 'Pickup Lat', 'Pickup Long', 'Destination Lat',
             'Destination Long'
           ]

In [ ]:
sns.relplot(data=train, x="Placement - Weekday (Mo = 1)",y="Time from Pickup to Arrival")

In [ ]:
sns.relplot(data=train, x="Platform Type",y="Time from Pickup to Arrival")

In [ ]:
sns.catplot(data=train, x="Personal or Business",y="Time from Pickup to Arrival")

In [ ]:
##Do some pairplots
sns.relplot(data=mergedtrain, x="Average_Rating", y="Time from Pickup to Arrival")

In [ ]:
clean = pd.read_pickle('clean_training')

In [ ]:
#convert time to seconds since midnight
def settime(time):
    split = time.split(':')
    tempsplit = split[2].split(' ')
    if int(split[0]) != 12:
        seconds = int(split[0])*3600 + int(split[1])*60 + int(tempsplit[0])
    elif tempsplit[1] == 'PM' and int(split[0]) == 12:
        seconds = int(split[0])*3600 + int(split[1])*60 + int(tempsplit[0])
    else:
        seconds = int(split[1])*60 + int(tempsplit[0])
    if tempsplit[1] == 'PM' and int(split[0]) != 12:
        seconds += 43200
    return seconds

In [ ]:

clean['Pickup_time'] = clean["Pickup_time"].apply(settime)

sns.scatterplot(data = clean.sort_values(by = 'Pickup_time'), x = "Pickup_time", y = "Travel_time")

In [ ]:
clean.head()

In [ ]:
clean.to_pickle('clean_training.pkl')

## Cleaning

In [ ]:
train.isnull().sum()

In [ ]:
train.isnull().sum()  /  train.shape[0]

In [ ]:
#Precipitation is mostly empty, so we drop it
mergedtrain = mergedtrain.drop(columns=useless)

In [ ]:
mergedtrain.head()

In [ ]:
mergedtrain.dtypes

In [ ]:
mergedtrain = mergedtrain.reindex(columns=["Rider Id", "No_Of_Orders", 'Age',"Average_Rating","No_of_Ratings", "Platform Type",
                                           "Personal or Business","Pickup - Weekday (Mo = 1)",
                                           "Pickup - Time","Distance (KM)","Temperature","Time from Pickup to Arrival"])

In [ ]:
mergedtrain.head()

In [ ]:
impute = mergedtrain["Temperature"]

In [ ]:
#Changes were lost but clean pickle was made